# **2. Theory**
In this part, we look at the underlying theory of the experiment.

## Basics
For a lossless and dispersionless material, there is a linear dependence of polarisation on electric field:

$$\textbf{P}(t)=\epsilon_0\chi^{(0)}\textbf{E}(t)$$


In a nonlinear medium, this dependence can be expanded to a power series.

$$\textbf{P}(t)=\epsilon_0\left[\chi^{(1)}\textbf{E}(t)+\chi^{(2)}\textbf{E}^2(t)+\chi^{(3)}\textbf{E}^3(t)+\ldots\right]$$

The nonlinear susceptibilities are usually taken to be constants for a dispersionless medium.

The second order polarisation due to interaction of the light beam with the medium is:

$$\textbf{P}^{(2)}(t) = \epsilon_0\chi^{(2)}\textbf{E}^2(t)$$

We express the electric field in terms of two frequency components as:

$$\textbf{E}(t) = E_1e^{-i\omega_1t}+E_2e^{-i\omega_2t}+c.c.$$

We can substitute this into the second-order polarisation and get

$$\textbf{P}^{(2)}(t) = 2\epsilon_0\chi^{(2)}(E_1E_1^*+E_2E_2^*)+\epsilon_0\chi^{(2)}(E_1^2e^{−2i\omega_1t} + E_2^2e^{−2i\omega_2t}+2E_1E_2e^{−i(\omega_1+\omega_2)t} + E_1E_2^∗e^{−i(\omega_1−\omega_2)t} + c.c.)$$


The nonlinear interaction of second-harmonic generation is due to the following terms:

$$P(2\omega_1) = \epsilon_0\chi^{(2)}E_1^2 $$

$$P(2\omega_2) = \epsilon_0\chi^{(2)}E_2^2 $$

The above two terms represnt the second-harmonic generation of signal waves of frequencies $\omega=2\omega_1$ and $\omega=2\omega_2$ from two pump waves of frequencies $\omega_1$ and $\omega_2$ respectively.

There are other second-order interactions such as sum-frequency generation, difference-frequency generation and optical rectification.

In sum-frequency generation (SFG), two photons of frequencies $\omega_1$ and $\omega_2$ combine to yield a photon of energy $\omega_3=\omega_1+\omega_2$.

In difference-frequency generation (DFG), a photon of higher frequency $\omega_1$ splits into two new photons of frequencies $\omega_2$ and $\omega_3=\omega_1-\omega_2$.

Finally, optical rectification occurs when the new frequency is the difference of one frequency with itself ($\omega_3 = \omega_1 - \omega_1 = 0$). This generates a static electric field.


### **Four-wave Mixing**

This procedure involves third-order nonlinear processes:

$$\textbf{E}(t) = E_1e^{−i\omega_1t} + E_2e^{−i\omega_2t} + E_3e^{−i\omega_3t} + c.c.$$

Substituting the above equation in the polarisation power series expansion yields $44$ distinct frequency terms. So we use the pump-degenerate four-wave mixing which is a third-order nonlinear process. This process is described by the following term:

$$P(2\omega_1 −\omega_2)=3\epsilon_0\chi^{(3)}E_1^2E_2^∗$$

In a pump-degenerate FWM nonlinear process, two photons (pump photons) at a frequency $\omega_1$ are converted into one photon (signal photon) of frequency $\omega_2$ and one photon (iddler photon) of frequency $\omega_3 = 2\omega_1-\omega_2$. We have:

$$\omega_p +\omega_p = (\omega_p +\delta\omega)+(\omega_p −\delta\omega) $$
$$=\omega_s + \omega_i$$

The pump degeneracy here implies that both the pump photons have the same energy. Most FWM processes are pump-degenerate.

## Wave Equation in Nonlinear Optics

Assume that the electric current density, $\textbf{J} = 0$. Maxwell's equations:
\begin{equation*}
\nabla\cdot \textbf{D} = 0
\label{eq:maxwell_1} \tag{1}
\end{equation*}

\begin{equation*}
\nabla\cdot \textbf{B} = 0
\label{eq:maxwell_2} \tag{2}
\end{equation*}

\begin{equation*}
\nabla\times \textbf{E} = -\frac{\partial\textbf{B}}{\partial t}
\label{eq:maxwell_3} \tag{3}
\end{equation*}

\begin{equation*}
\nabla\times \textbf{H} = \frac{\partial\textbf{D}}{\partial t}
\label{eq:maxwell_4} \tag{4}
\end{equation*}

<!-- $$\nabla\cdot \textbf{D} = 0$$
$$\nabla\cdot \textbf{B} = 0$$ -->
<!-- 
$$\nabla\times \textbf{E} = -\frac{\partial\textbf{B}}{\partial t}$$
$$\nabla\times \textbf{H} = \frac{\partial\textbf{D}}{\partial t}$$ -->

\begin{equation*}
\Rightarrow\textbf{B} = \mu_0\mu\textbf{H}
\label{eq:maxwell_5} \tag{5}
\end{equation*}

\begin{equation*}
\Rightarrow\textbf{D} = \epsilon_0\epsilon\textbf{E} + P^{NL}
\label{eq:maxwell_6} \tag{6}
\end{equation*}

<!-- $$\Rightarrow\textbf{B} = \mu_0\mu\textbf{H}$$
$$\Rightarrow\textbf{D} = \epsilon_0\epsilon\textbf{E} + P^{NL}$$ -->

$\textbf{B}$ is the magnetic induction field, $\textbf{D}$ is the electric displacement field, and $\textbf{H}$ is the magnetic intensity field.

To obtain the nonlinear optical wave equation, substitute $\eqref{eq:maxwell_5}$ into $\eqref{eq:maxwell_3}$ and take the curl of both the sides to obtain:

\begin{equation*}
\nabla\textbf{E}_n-\frac{\epsilon_n\mu_n}{c^2}\frac{\partial^2\textbf{E}_n}{\partial t^2} = \frac{\mu_n}{\epsilon_0c^2}\frac{\partial^2\textbf{P}_n^{NL}}{\partial t^2}
\label{eq:NL_Wave_Equation} \tag{7}
\end{equation*}

Here, the subscript $n$ is used to denote each frequency component of the field.

### Coupled-amplitude Equations

<insert pic of coupled-amplitude equations block and signals>
    
 When $\textbf{P}^{NL} = 0$, the solution to the nonlinear optical wave equation is a plane wave propagating in the $z$ direction. This wave is described by the following equation:
 
\begin{equation*}
\textbf{E}_n(z, t) = A_ne^{i(k_nz-\omega_nt)} + c.c.
\label{eq:coupled_amp} \tag{8}
\end{equation*}
    
When $\textbf{P}^{NL}$ is nonzero and not very large, we can take $\eqref{eq:coupled_amp}$ as the solution to the nonlinear optical wave equation with the nonlinear source included. We represent this nonlinear source as:
   
\begin{equation*}
\textbf{P}_n(z, t) = P_ne^{-i\omega_nt} + c.c.
\label{eq:NL_Source} \tag{9}
\end{equation*}
    
Now we represent the pump, signal and idler terms for $P_n$:
    
\begin{equation*}
P_p = 3\epsilon_0\chi^{(3)}A_p^∗A_sA_ie^{i(k_sz+k_iz-k_pz)} + c.c.
\label{eq:NL_Source_pump} \tag{10}
\end{equation*}
    
\begin{equation*}
P_s = 3\epsilon_0\chi^{(3)}A_p^2A_i^*e^{i(2k_pz - k_iz)} + c.c.
\label{eq:NL_Source_signal} \tag{11}
\end{equation*}
    
\begin{equation*}
P_i = 3\epsilon_0\chi^{(3)}A_p^2A_s^*e^{i(2k_pz - k_sz)} + c.c.
\label{eq:NL_Source_idler} \tag{12}
\end{equation*}
    
Substituting the terms into the nonlinear optical wave equation, we finally get,
    
\begin{equation*}
\frac{d^2A_i}{dz^2}+2ik_i\frac{dA_i}{dz} = \frac{-3\mu_i\omega_i^2\chi^{(3)}}{c^2}A_p^2A_s^*e^{i(2k_pz-k_sz-k_iz)}
\label{eq:NL_Source_final} \tag{13}
\end{equation*}
    
Since the second derivative varies very slowly, we can drop the second derivative. This gives us:
\begin{equation*}
\left|\frac{d^2A_i}{dz^2}\right| << \left|k_i\frac{dA_i}{dz}\right|
\label{eq:SVAA} \tag{14}
\end{equation*}
    
Now we define the wave vector mismatch factor $\delta k$:
\begin{equation*}
\Delta k \equiv 2k_p - k_s - k_i
\label{eq:phase_mismatch} \tag{15}
\end{equation*}
    
Making use of $\eqref{eq:phase_mismatch}$ and $\eqref{eq:SVAA}$ in $\eqref{eq:NL_Source_final}$, we get the coupled-amplitude equation for the idler frequency:
    
\begin{equation*}
\frac{dA_i}{dz} = \frac{3i\mu_i\omega_i\chi^{(3)}}{2n_ic}A_p^2A_s^*e^{i\Delta kz}
\label{eq:coupled_idler} \tag{16}
\end{equation*}
    
Similarly, we obtain the coupled-amplitude equations for the pump and signal frequencies:
    
\begin{equation*}
\frac{dA_p}{dz} = \frac{3i\mu_p\omega_p\chi^{(3)}}{2n_pc}A_p^*A_sA_ie^{-i\Delta kz}
\label{eq:coupled_pump} \tag{17}
\end{equation*}
    
\begin{equation*}
\frac{dA_s}{dz} = \frac{3i\mu_s\omega_s\chi^{(3)}}{2n_sc}A_p^2A_i^*e^{i\Delta kz}
\label{eq:coupled_signal} \tag{18}
\end{equation*}
    
The above three equations describe the FWM entirely.

### Phase Matching

In order to solve the coupled-amplitude equations, we use the undepleted pump approximation. As a result of this, the pump and signal wave amplitudes, $A_p$ and $A_s$ remain constant.

When we have the condition of perfect phase matching ($\Delta k = 0$), the idler coupled-amplitude is solved with:

\begin{equation*}
A_i = \frac{3i\mu_i\omega_i\chi^{(3)}}{2n_ic}A_p^2A_s^*L
\label{eq:idler_soln} \tag{19}
\end{equation*}

where $L$ is the length of the waveguide.

When $\Delta k \neq 0$, the solution to the idler coupled-amplitude becomes:

\begin{equation*}
A_i = \frac{3i\mu_i\omega_i\chi^{(3)}}{2n_ic}A_p^{*2}A_s\int^{L}_{0
} e^{i\Delta kz} dz
\label{eq:idler_soln_mismatch} \tag{20}
\end{equation*}

\begin{equation*}
=\frac{3i\mu_i\omega_i\chi^{(3)}}{2n_ic}A_p^{*2}A_s\left(\frac{e^{i\Delta kL} - 1}{i\Delta k}\right)
\label{eq:idler_soln_mismatch2} \tag{21}
\end{equation*}

Squaring the modulus of the phase-matching term:
\begin{equation*}
\DeclareMathOperator{\sinc}{sinc}
\left|\frac{e^{i\Delta kL} - 1}{i\Delta k}\right|^2 = L^2\frac{\sin^2(\Delta kL/2)}{(\Delta kL/2)^2} \equiv L^2\sinc^2(\Delta kL/2)
\label{eq:mod_phase_match} \tag{22}
\end{equation*}

We can define a length variable which depends on the phase matching. This 

\begin{equation*}
L_{coh} = \frac{2}{\Delta k}
\label{eq:L_coh} \tag{23}
\end{equation*}

## Zero-index Materials

Classical optics induces the notion that the refractive index of materials is greater than or equal to one. A refractive index lower than one would result in strange phenomena like a phase velocity exceeding that of light.

### Properties of Zero-index Materials

Zero-index materials imply a infinite phase velocity resulting in a phaseless propagation. We can confirm this by looking at 

\begin{equation*}
v_p = c/n
\label{eq:zero_index} \tag{24}
\end{equation*}

where $v_p$ is the phase velocity, $c$ is the speed of light and $n$ is the refractive index of the material.
To overcome this violation, special relativity limits the signal velocity to the speed of light. The signal velocity is equal to the group velocity in most cases. So,

\begin{equation*}
v_g = \frac{c}{n + \omega\frac{\partial n}{\partial\omega}}
\label{eq:sig_velo} \tag{25}
\end{equation*}

where $\frac{\partial n}{\partial\omega}$ describes the dispersion. When the dispersion is zero, then the group velocity equals the phase velocity. 